In [2]:
import numpy as np

In [2]:
from sklearn.metrics.pairwise import euclidean_distances

In [3]:
positions = np.arange(10).reshape(-1,2)

In [4]:
positions

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [5]:
dist_matrix = euclidean_distances(positions)
dist_matrix

array([[ 0.        ,  2.82842712,  5.65685425,  8.48528137, 11.3137085 ],
       [ 2.82842712,  0.        ,  2.82842712,  5.65685425,  8.48528137],
       [ 5.65685425,  2.82842712,  0.        ,  2.82842712,  5.65685425],
       [ 8.48528137,  5.65685425,  2.82842712,  0.        ,  2.82842712],
       [11.3137085 ,  8.48528137,  5.65685425,  2.82842712,  0.        ]])

In [6]:
dist_matrix[dist_matrix<5] = 0

In [7]:
dist_matrix

array([[ 0.        ,  0.        ,  5.65685425,  8.48528137, 11.3137085 ],
       [ 0.        ,  0.        ,  0.        ,  5.65685425,  8.48528137],
       [ 5.65685425,  0.        ,  0.        ,  0.        ,  5.65685425],
       [ 8.48528137,  5.65685425,  0.        ,  0.        ,  0.        ],
       [11.3137085 ,  8.48528137,  5.65685425,  0.        ,  0.        ]])

In [14]:
states = np.load('state.npz')
X = states['arr_0'][0]
A = states['arr_0'][1]
rl_ind = states['arr_0'][2]
X.shape, A.shape, rl_ind

((36, 9),
 (36, 36),
 array([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35]))

In [15]:
A

array([[1., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [31]:
from spektral.layers import GraphConv
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.backend import gather, squeeze
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


## First trail

In [38]:
class GraphicEncoder():
    def __init__(self, F):
        self.model = self.build_model(F)

    def build_model(self,F):
        X_in = Input(shape=(F,), name='X_in')
        A_in = Input(shape=(None,), name='A_in')
        RL_indice = Input(shape=(None,),batch_size = 1, name='RL_indice',dtype='int32')

        
        ### Graphic convolution
        
        x = GraphConv(32, activation='relu',name='gcn1')([X_in, A_in])
        x = GraphConv(32, activation='relu',name='gcn2')([x, A_in])
        x = Lambda(lambda x: gather(x,RL_indice),name='slice')(x)
        
        x = Lambda(lambda x: squeeze(x,0),name='squeeze')(x)
        
        ### Policy network
        
        x = Dense(32,activation='relu',name='policy_1')(x)
        x = Dense(16,activation='relu',name='policy_2')(x)
        x = Dense(3, activation='relu',name='policy_3')(x)
        
        
        model = Model(inputs = [X_in,A_in,RL_indice], outputs=x)
        print(model.summary())
        return model
    
    
feature_size = 9
rl_model = GraphicEncoder(feature_size)


Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_in (InputLayer)               [(None, 9)]          0                                            
__________________________________________________________________________________________________
A_in (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
gcn1 (GraphConv)                (None, 32)           320         X_in[0][0]                       
                                                                 A_in[0][0]                       
__________________________________________________________________________________________________
gcn2 (GraphConv)                (None, 32)           1056        gcn1[0][0]                

In [46]:
from gym.spaces.box import Box
from gym.spaces import Discrete
from gym.spaces.dict import Dict

In [52]:
from ray.rllib.models.tf.tf_modelv2 import TFModelV2

W0512 21:19:35.218683 4537492928 deprecation.py:323] From /anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [53]:
F = 9
N = 40

states = Box(low=-np.inf, high=np.inf, shape=(N,F), dtype=np.float32)
adjacency = Box(low=0, high=1, shape = (N,N), dtype=np.int32)
mask = Box(low=0, high=1, shape = (N,), dtype=np.int32)

obs_space = Dict({'states':states,'adjacency':adjacency,'mask':mask})
act_space = Box(low=0, high=1, shape = (N,), dtype=np.int32)

## Second trail

In [73]:
from spektral.layers import GraphConv
from tensorflow.keras.layers import Input, Dense, Lambda, Multiply, Reshape, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

class GraphicPolicy(TFModelV2):
    def __init__(self, N,F, obs_space, action_space, num_outputs=3, model_config=None, name='graphic_policy'):
        super(GraphicPolicy,self).__init__(obs_space, action_space, num_outputs, model_config, name)
        self.base_model = self.build_model(N,F,num_outputs)
        self.register_variables(self.base_model.variables)
        
    def build_model(self,N,F,num_outputs):
        X_in = Input(shape=(N,F), name='X_in')
        A_in = Input(shape=(N,N), name='A_in')
        RL_indice = Input(shape=(N), name='mask')
        
        ### Graphic convolution
        
        x = GraphConv(32, activation='relu',name='gcn1')([X_in, A_in])
        x = GraphConv(32, activation='relu',name='gcn2')([x, A_in])

        
        ### Policy network
        
        x1 = Dense(32,activation='relu',name='policy_1')(x)
        x2 = Dense(16,activation='relu',name='policy_2')(x1)
        
    
        ###  Action and filter
        x3 = Dense(num_outputs, activation='relu',name='policy_3')(x2)
        mask = Reshape((N,1),name='expend_dim')(RL_indice)
        out = Multiply(name='filter')([x3,mask])
        
        
        #### Value out
        x2 = Flatten(name='flatten')(x2)
        value = Dense(1,activation='relu',name='value_out')(x2)
        

        model = Model(inputs = [X_in,A_in,RL_indice], outputs=[out,value])
        print(model.summary())
        return model
    
    def forward(self, input_dict, state, seq_lens):
        obs = input_dic['obs']
        model_out, self._value_out = self.base_model({X_in:obs['states'],A_in:obs['adjacency'],RL_indice:obs['mask']})        
        return model_out,state
        
    def value_function(self):
        return tf.reshape(self._value_out, [-1])
    
model_config = {}
rl_model = GraphicPolicy(N,F, obs_space, act_space)

Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_in (InputLayer)               [(None, 40, 9)]      0                                            
__________________________________________________________________________________________________
A_in (InputLayer)               [(None, 40, 40)]     0                                            
__________________________________________________________________________________________________
gcn1 (GraphConv)                (None, 40, 32)       320         X_in[0][0]                       
                                                                 A_in[0][0]                       
__________________________________________________________________________________________________
gcn2 (GraphConv)                (None, 40, 32)       1056        gcn1[0][0]                